# OpenAI ChatCompletion API 를 비동기/병렬 응답 취합

병렬 요청/응답 취합
- OpenAI Cookbook 의 `api_request_parallel_processor.py` 를 이용
  - 요청 파일에 대해 비동기/병렬 요청
  - 응답 결과를 파일에 저장

# 응답 파일 가공

입력: API 응답이 담긴 JSONL 파일

출력: 변환 후 목적에 맞는 결과 파일

In [1]:
import pandas as pd
# from datasets import Features, ClassLabel, Dataset, DatasetDict
import json
import re

In [2]:
respose_file_path = r'../local_data/api_responses_for_annotated_dataset.jsonl'

In [3]:
in_fname =  respose_file_path
datas = []
request_ids = []
# results = {}
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if isinstance(data[1] , dict):
            # results[i] = data
            datas.append(data[1]['choices'][0]['message']['content'])
            request_ids.append(data[2]['request_id'])

In [4]:
df = pd.DataFrame({'request_id': request_ids, 'response': datas})
df['response'] = df['response'].str.split('\n')
df.sort_values(by='request_id', inplace=True)

# response 컬럼을 펼쳐서 새로운 DataFrame 생성
exploded_df = df.explode('response').reset_index(drop=True)

# # "->" 가 포함된 response만 남기기 (한건 제외됨)
# exploded_df = exploded_df[exploded_df['response'].str.contains('->')]

**주의**

- 제외되는 것들이 있음: 원본 4,239 (응답 결과: 4,201)

In [5]:
# 수정된 serial_no, category, keywords 추출 함수
def extract_columns_with_list(response):
    # '.'이나 ':' 기준으로 첫 구분자까지 split
    match = re.match(r'(\d+)\s*[.:]', response)
    if not match:
        return '', '', []
    
    # serial_no 추출
    serial_no = match.group(1).strip()
    
    # 나머지 부분 추출
    rest = response[match.end():].strip()
    
    rest = rest.split(':')
    
    category = rest[0].strip()
    keywords = []
    if len(rest) > 1:
        keywords = rest[1:]
    
    return serial_no, category, keywords



In [6]:
# apply를 사용하여 수정된 컬럼 생성
exploded_df[['serial_no', 'category', 'keywords']] = exploded_df['response'].apply(lambda x: pd.Series(extract_columns_with_list(x)))

In [7]:
exploded_df['serial_no'] = exploded_df['serial_no'].astype(int)

In [8]:
# request_id를 'tester', 'trial_idx', 'call_id'로 분리
exploded_df[['tester', 'trial_idx', 'call_id']] = exploded_df['request_id'].str.split(':', expand=True)

# trial_idx와 call_id를 숫자형으로 변환
exploded_df['trial_idx'] = exploded_df['trial_idx'].astype(float).astype(int)
exploded_df['call_id'] = exploded_df['call_id'].astype(int)

In [10]:
# 필요 없는 request_id 컬럼 제거
exploded_df = exploded_df.drop(columns=['request_id'])


In [11]:
exploded_df.to_csv('../local_data/annotated_dataset.csv', index=False)

# 원본 엑셀과 병합

In [12]:
raw_file_path = r'../local_data/(2024-04-29)구글 디스커버_데이터_수집.xlsx'

In [13]:
all_sheets = pd.read_excel(raw_file_path, sheet_name=None)
# new_cols = ['url-title', 'source', 'issue-dt', 'title', 'url', 'precision', 'freshness',
#        'satisfaction', 'trial-index', 'dt']

In [14]:
merged_sheets = {}

In [15]:
for key in all_sheets.keys():
    tmp_df = exploded_df.query('tester == @key')
    tmp_df = pd.merge(all_sheets[key].reset_index(), tmp_df[['category', 'keywords', 'serial_no']], left_on = 'index', right_on=tmp_df.serial_no-1, how='left')
    merged_sheets[key] = tmp_df

In [16]:
with pd.ExcelWriter('../local_data/(2024-04-29)구글 디스커버_데이터_수집 카테고리_키워드 연결.xlsx') as writer:
    for sheet_name, sheet_df in merged_sheets.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

----